# Cadherin distribution simulation (control parameters)

In [11]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import FancyBboxPatch
from IPython.display import HTML

plt.rcParams['animation.embed_limit'] = 200

# Parameters
SCALE             = 2    # dimensionless scale factor (pixels per µm)
Nx = Ny           = 60 * SCALE   # raster size (pixels)
dt                = 0.1          # time step (s)
STEPS             = 1310         # total integration steps for control sample
SAVE_EVERY        = 2            # render every N steps
MAX_PER_PIXEL     = 14           # occupancy cap per pixel (particles)

# Diffusion: literature value for E-cadherin (Erami et al., 2016), in µm²/s.
D_LAT_UM2_S       = 0.0048       # µm²/s
D_LAT             = D_LAT_UM2_S * (SCALE ** 2)   # pixel²/s

# Two-phase advection visualization controls
PHASE1_STEPS      = 630          # end of phase 1 (ventral flow, control)
PCLIP             = 90           # percentile for colour stretch (each frame)
TOP_PERC          = 95           # overlay ≥TOP_PERC percentile if not None

# Cell-cell contact area definition
cx_dom = cy_dom = 30 * SCALE

# 2D capsule geometry
CAPS_R       = 13 * SCALE
CAPS_HALF_L  = 16 * SCALE

# Inner corner points along the capsule's center line (used for distance queries)
A = np.array([cx_dom, cy_dom - (CAPS_HALF_L - CAPS_R)])  # top inner corner
B = np.array([cx_dom, cy_dom + (CAPS_HALF_L - CAPS_R)])  # bottom inner corner
BA      = B - A
BA_dot  = BA @ BA

# Functions
def signed_dist_capsule(P):
    """Return +distance inside, −outside for a vertical capsule."""
    pa = P - A
    h  = np.clip((pa @ BA) / BA_dot, 0.0, 1.0)[:, None]
    closest = A + h * BA
    return CAPS_R - np.linalg.norm(P - closest, axis=1)  # >0 inside

def reflect_capsule(P_old, P_new):
    """Specular reflection at the capsule boundary."""
    out = signed_dist_capsule(P_new) < 0
    if not np.any(out):
        return P_new
    P_out = P_new[out]
    pa    = P_out - A
    h     = np.clip((pa @ BA) / BA_dot, 0.0, 1.0)[:, None]
    nearest = A + h * BA
    n_vec   = P_out - nearest
    n_hat   = n_vec / np.linalg.norm(n_vec, axis=1, keepdims=True)
    d   = P_out - P_old[out]
    d_n = (d * n_hat).sum(axis=1, keepdims=True) * n_hat
    P_new[out] = P_old[out] + d - 2 * d_n
    return P_new

def enforce_occupancy(P_old, P_new, max_occ=MAX_PER_PIXEL):
    """Reject moves into pixels whose occupancy would exceed max_occ."""
    ix = np.clip(np.rint(P_new[:, 0]).astype(int), 0, Nx - 1)
    iy = np.clip(np.rint(P_new[:, 1]).astype(int), 0, Ny - 1)
    flat = iy * Nx + ix
    uniq, cnts = np.unique(flat, return_counts=True)
    crowded = uniq[cnts > max_occ]
    if crowded.size:
        mask = np.isin(flat, crowded)
        P_new[mask] = P_old[mask]
    return P_new

# Initial condition: clustered cadherin patch with holes
cx = cy = Nx // 2
PATCH_RAD = 8 * SCALE
Y, X = np.indices((Ny, Nx))
circ = (X - cx) ** 2 + (Y - cy) ** 2 <= PATCH_RAD ** 2

# Capsule mask
mask_caps = signed_dist_capsule(np.column_stack([X.ravel(), Y.ravel()])).reshape(Ny, Nx) > 0

# Three small “holes”
rng_hole = np.random.default_rng(27)
hole = np.zeros_like(circ, bool)
for _ in range(3):
    while True:
        hx = rng_hole.integers(cx - PATCH_RAD, cx + PATCH_RAD + 1)
        hy = rng_hole.integers(cy - PATCH_RAD, cy + PATCH_RAD + 1)
        if (hx - cx) ** 2 + (hy - cy) ** 2 <= PATCH_RAD ** 2:
            break
    hole |= (X - hx) ** 2 + (Y - hy) ** 2 <= (3 ** 2)

in_domain = circ & mask_caps & ~hole

# Random-size clusters (primary samples + local jitter)
N_CLUSTERS       = 20
R_MIN, R_MAX     = 2 * SCALE, 5 * SCALE
DENSITY_FACTOR   = 8
MULTIPLIER_LOCAL = 1
rng_cls  = np.random.default_rng(32)
rng_mul  = np.random.default_rng(32)

pts = []
for _ in range(N_CLUSTERS):
    while True:
        cx_i = rng_cls.integers(cx - PATCH_RAD, cx + PATCH_RAD + 1)
        cy_i = rng_cls.integers(cy - PATCH_RAD, cy + PATCH_RAD + 1)
        if (cx_i - cx) ** 2 + (cy_i - cy) ** 2 <= PATCH_RAD ** 2:
            break
    R = rng_cls.uniform(R_MIN, R_MAX)
    n_primary = int(DENSITY_FACTOR * (R / SCALE) ** 2)
    r     = R * np.sqrt(rng_cls.random(n_primary))
    theta = rng_cls.random(n_primary) * 2 * np.pi
    xs0   = cx_i + r * np.cos(theta)
    ys0   = cy_i + r * np.sin(theta)
    keep = in_domain[np.clip(ys0.astype(int), 0, Ny - 1),
                     np.clip(xs0.astype(int), 0, Nx - 1)]
    primary = np.column_stack([xs0[keep], ys0[keep]])
    pts.extend(primary)

    if MULTIPLIER_LOCAL:
        jitter   = rng_mul.random((len(primary) * MULTIPLIER_LOCAL, 2)) - 0.5
        expanded = np.repeat(primary, MULTIPLIER_LOCAL, axis=0) + jitter
        keep2 = in_domain[np.clip(expanded[:, 1].astype(int), 0, Ny - 1),
                          np.clip(expanded[:, 0].astype(int), 0, Nx - 1)]
        pts.extend(expanded[keep2])

pts = np.asarray(pts, float)
pts = enforce_occupancy(pts, pts)
rng_step = np.random.default_rng(0)

# Velocity fields
# Phase 1 (ventral flow)
vx1 = np.zeros((Ny, Nx))
vy1 = np.zeros_like(vx1)
vy1[:] = -0.123 * SCALE

# Phase 2 (chiral flow) — NOTE: this uses Nx on y-slices as in your original;
# if you meant dorsal/ventral splits, change Nx → Ny on vy2 slices.
vx2 = np.zeros_like(vx1)
vy2 = np.zeros_like(vx1)

# x-component: opposite signs on left/right halves
vx2[:Nx//3+3] =  +0.087 * SCALE
vx2[Nx//3+3:] = -0.052 * SCALE

# y-component (kept verbatim from your code; see note above)
vy2[Nx//3+3:] = -0.052 * SCALE
vy2[:Nx//3+3] = -0.034 * SCALE

def vel_at(xf, yf, vx, vy):
    xi = np.clip(np.rint(xf).astype(int), 0, Nx - 1)
    yi = np.clip(np.rint(yf).astype(int), 0, Ny - 1)
    return vx[yi, xi], vy[yi, xi]

def step(k, P):
    """Euler–Maruyama step with affine correction."""
    vx, vy = (vx1, vy1) if k < PHASE1_STEPS else (vx2, vy2)
    vx_p, vy_p = vel_at(P[:, 0], P[:, 1], vx, vy)
    V = np.column_stack((vx_p, vy_p))

    Pc = P.mean(0);  Vc = V.mean(0)
    R  = P - Pc;     Vr = V - Vc
    RTR = R.T @ R
    J = (Vr.T @ R) @ np.linalg.inv(RTR) if np.linalg.cond(RTR) < 1e12 else np.zeros((2, 2))

    Pn = P + dt * (Vc + (J @ R.T).T) \
           + np.sqrt(2 * D_LAT * dt) * rng_step.standard_normal(P.shape)

    Pn = reflect_capsule(P, Pn)
    Pn = enforce_occupancy(P, Pn)
    return Pn

def raster(P):
    """Rasterize particle counts m[y, x] (origin='lower')."""
    m = np.zeros((Ny, Nx))
    xi = np.clip(np.rint(P[:, 0]).astype(int), 0, Nx - 1)
    yi = np.clip(np.rint(P[:, 1]).astype(int), 0, Ny - 1)
    np.add.at(m, (yi, xi), 1)
    return m

# Visualization set-up
fig = plt.figure(figsize=(10, 4))
gs  = fig.add_gridspec(1, 3, width_ratios=[1.2, 1.2, 1.2])
ax0, ax1, ax2 = [fig.add_subplot(gs[i]) for i in range(3)]

m0   = raster(pts)
nz0  = m0[m0 > 0]
vmax0 = np.percentile(nz0, PCLIP) if nz0.size else 1

im0 = ax0.imshow(m0, origin='lower', cmap='inferno', vmin=1, vmax=vmax0)
ax0.set_title('simulated cadherin localization'); ax0.axis('off')

sel_offsets = np.array([-7, 7])
sel_x_px    = cx + sel_offsets * SCALE
REF_SHIFT   = 7
y_coords    = (np.arange(Ny) - cy) / SCALE + REF_SHIFT

lines = [ax1.plot([], [], label=f'x={off:+d}')[0] for off in sel_offsets]
ax1.set_title('HMR-1 signal near the left or right surface (y = 0: cleavage furrow)')
ax1.set_xlabel('normalized occupancy (Σ_y = 1)')
ax1.set_ylabel('y  (0 ≡ cy – 7 px)')
ax1.invert_yaxis(); ax1.legend()
ax1.axhline(0, color='black', ls='--', lw=1)

for ax in (ax0, ax2):
    patch = FancyBboxPatch((cx_dom - CAPS_R + 0.5, cy_dom - CAPS_HALF_L + 0.5),
                           2 * CAPS_R, 2 * CAPS_HALF_L,
                           boxstyle=f"round,pad=0,rounding_size={CAPS_R}",
                           edgecolor='white', facecolor='none', lw=1.5, ls=':')
    ax.add_patch(patch); ax.set_aspect('equal')

im2base = ax2.imshow(m0, origin='lower', cmap='inferno', vmin=1, vmax=vmax0)
if TOP_PERC is not None:
    im2 = ax2.imshow(np.where(mask_caps, 0, np.nan), origin='lower',
                     cmap='Reds', vmin=0, vmax=1, alpha=0.6)
else:
    im2 = ax2.imshow(np.full_like(m0, np.nan, dtype=float), origin='lower',
                     cmap='Reds', vmin=0, vmax=1, alpha=0.6)

ax2.set_title('vlines: positions used for intensity profile'); ax2.axis('off')
ax2.axvline(cx - 7 * SCALE, color='white', ls='--', lw=1)
ax2.axvline(cx + 7 * SCALE, color='white', ls='--', lw=1)
ax2.axhline(cy - 7 * SCALE, color='white', ls='--', lw=1)

skip = 4 * SCALE
xq, yq = np.arange(0, Nx, skip), np.arange(0, Ny, skip)
Xq, Yq = np.meshgrid(xq, yq)
quiv = ax0.quiver(Xq, Yq, vx1[Yq, Xq], vy1[Yq, Xq], scale=5, headwidth=6, headlength=8, color = (1,1,1,0.6), linewidth =0.0, zorder =5)

fig.tight_layout()
title_text = fig.suptitle('')

def update(frame):
    """Integrate SAVE_EVERY steps, then refresh rasters/overlays."""
    global pts
    k = frame * SAVE_EVERY
    for _ in range(SAVE_EVERY):
        pts = step(k, pts)

    m = raster(pts)
    nz = m[m > 0]
    vmax = np.percentile(nz, PCLIP) if nz.size else 1

    im0.set_data(m);      im0.set_clim(1, vmax)
    im2base.set_data(m);  im2base.set_clim(1, vmax)

    vx, vy = (vx1, vy1) if k < PHASE1_STEPS else (vx2, vy2)
    quiv.set_UVC(vx[Yq, Xq], vy[Yq, Xq])

    for l, x in zip(lines, sel_x_px):
        col = m[:, x].astype(float)
        total = col.sum()
        l.set_data(col / total if total else col, y_coords)

    ax1.set_xlim(0, 1)
    ax1.set_ylim(y_coords.min(), y_coords.max())

    if TOP_PERC is not None:
        thresh = np.percentile(m[mask_caps], TOP_PERC)
        im2.set_data(np.where(mask_caps & (m >= thresh), 1, np.nan))
    else:
        im2.set_data(np.full_like(m, np.nan, dtype=float))

    title_text.set_text(f"t = {k*dt:.1f} s   ({'phase-1' if k < PHASE1_STEPS else 'phase-2'})")
    return [im0, im2base, quiv, im2, *lines, title_text]

# Build animation
frames = np.arange(0, STEPS // SAVE_EVERY + 1)
ani = FuncAnimation(fig, update, frames=frames, blit=True, interval=1000*dt)


# -------- MP4 export --------
fps = 30
mp4_path = 'cadherin_simulationcontrolnew.mp4'
try:
    from matplotlib.animation import FFMpegWriter
    writer = FFMpegWriter(
        fps=fps,
        codec='libx264',
        bitrate=3000,
        extra_args=['-pix_fmt', 'yuv420p']  # broad compatibility (PowerPoint, QuickTime)
    )
    ani.save(mp4_path, writer=writer, dpi=200)
    print(f"✓ Saved MP4 to {mp4_path}  (fps={fps}, dpi=200)")
except Exception as e:
    # fallback: try string writer if FFMpegWriter import succeeded but codec/path not found
    try:
        ani.save(mp4_path, writer='ffmpeg', fps=fps, dpi=200, extra_args=['-pix_fmt', 'yuv420p'])
        print(f"✓ Saved MP4 to {mp4_path}  (fps={fps}, dpi=200)")
    except Exception as ee:
        print("✗ Could not write MP4. Install ffmpeg (e.g., `conda install -c conda-forge ffmpeg`) and retry.")
        print("  Error:", ee)


✓ Saved MP4 to cadherin_simulationcontrolnew.mp4  (fps=30, dpi=200)


# Cadherin distribution simulation (control, in line animation)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import FancyBboxPatch
from IPython.display import HTML

plt.rcParams['animation.embed_limit'] = 200

# Parameters
SCALE             = 2    # dimensionless scale factor (pixels per µm)
Nx = Ny           = 60 * SCALE   # raster size (pixels)
dt                = 0.1          # time step (s)
STEPS             = 1310         # total integration steps for control sample
SAVE_EVERY        = 2            # render every N steps
MAX_PER_PIXEL     = 14           # occupancy cap per pixel (particles)

# Diffusion: literature value for E-cadherin (Erami et al., 2016), in µm²/s.
D_LAT_UM2_S       = 0.0048       # µm²/s
D_LAT             = D_LAT_UM2_S * (SCALE ** 2)   # pixel²/s

# Two-phase advection visualization controls
PHASE1_STEPS      = 630          # end of phase 1 (ventral flow, control)
PCLIP             = 90           # percentile for colour stretch (each frame)
TOP_PERC          = 95           # overlay ≥TOP_PERC percentile if not None


# Cell-cell contact area definition
# Position of the cell-cell contact area
cx_dom = cy_dom = 30 * SCALE

# 2D capsule geometry (CAPS_R: radius of half circle, CAPS_HALF_L: half height of the total area)
CAPS_R       = 13 * SCALE
CAPS_HALF_L  = 16 * SCALE

# Inner corner points along the capsule's center line (used for distance queries)
A = np.array([cx_dom, cy_dom - (CAPS_HALF_L - CAPS_R)])  # top inner corner
B = np.array([cx_dom, cy_dom + (CAPS_HALF_L - CAPS_R)])  # bottom inner corner
BA      = B - A
BA_dot  = BA @ BA


# Functions
def signed_dist_capsule(P):
    """
    Return +distance inside, −outside for a vertical capsule defined by corners A, B
    and radius CAPS_R. Used to (a) mask the domain and (b) compute reflection normals.
    """
    pa = P - A
    h  = np.clip((pa @ BA) / BA_dot, 0.0, 1.0)[:, None]
    closest = A + h * BA
    return CAPS_R - np.linalg.norm(P - closest, axis=1)  # >0 inside


def reflect_capsule(P_old, P_new):
    """
    Specular reflection at the capsule boundary.
    Any proposed step that exits the domain is mirrored across the local normal.
    """
    out = signed_dist_capsule(P_new) < 0
    if not np.any(out):
        return P_new
    P_out = P_new[out]
    pa    = P_out - A
    h     = np.clip((pa @ BA) / BA_dot, 0.0, 1.0)[:, None]
    nearest = A + h * BA
    n_vec   = P_out - nearest
    n_hat   = n_vec / np.linalg.norm(n_vec, axis=1, keepdims=True)

    d   = P_out - P_old[out]
    d_n = (d * n_hat).sum(axis=1, keepdims=True) * n_hat
    P_new[out] = P_old[out] + d - 2 * d_n
    return P_new


def enforce_occupancy(P_old, P_new, max_occ=MAX_PER_PIXEL):
    """
    Reject moves into pixels whose occupancy would exceed max_occ.
    Biologically: prevents unphysical local pile-ups of cadherin foci.
    """
    ix = np.clip(np.rint(P_new[:, 0]).astype(int), 0, Nx - 1)
    iy = np.clip(np.rint(P_new[:, 1]).astype(int), 0, Ny - 1)
    flat = iy * Nx + ix
    uniq, cnts = np.unique(flat, return_counts=True)
    crowded = uniq[cnts > max_occ]
    if crowded.size:
        mask = np.isin(flat, crowded)
        P_new[mask] = P_old[mask]
    return P_new


# Initial condition: clustered cadherin patch with holes
cx = cy = Nx // 2                     # patch center (pixels)
PATCH_RAD = 8 * SCALE                 # patch radius (pixels)

Y, X = np.indices((Ny, Nx))
circ = (X - cx) ** 2 + (Y - cy) ** 2 <= PATCH_RAD ** 2

# Capsule mask (True inside domain)
mask_caps = signed_dist_capsule(np.column_stack([X.ravel(), Y.ravel()])).reshape(Ny, Nx) > 0

# Three small “holes” representing local gaps in coverage
rng_hole = np.random.default_rng(27)
hole = np.zeros_like(circ, bool)
for _ in range(3):
    while True:
        hx = rng_hole.integers(cx - PATCH_RAD, cx + PATCH_RAD + 1)
        hy = rng_hole.integers(cy - PATCH_RAD, cy + PATCH_RAD + 1)
        if (hx - cx) ** 2 + (hy - cy) ** 2 <= PATCH_RAD ** 2:
            break
    hole |= (X - hx) ** 2 + (Y - hy) ** 2 <= (3 ** 2)

in_domain = circ & mask_caps & ~hole

# Random-size clusters (primary samples + local jitter)
N_CLUSTERS       = 20
R_MIN, R_MAX     = 2 * SCALE, 5 * SCALE
DENSITY_FACTOR   = 8
MULTIPLIER_LOCAL = 1
rng_cls  = np.random.default_rng(32)
rng_mul  = np.random.default_rng(32)

pts = []
for _ in range(N_CLUSTERS):
    # choose a cluster center within the patch
    while True:
        cx_i = rng_cls.integers(cx - PATCH_RAD, cx + PATCH_RAD + 1)
        cy_i = rng_cls.integers(cy - PATCH_RAD, cy + PATCH_RAD + 1)
        if (cx_i - cx) ** 2 + (cy_i - cy) ** 2 <= PATCH_RAD ** 2:
            break

    R = rng_cls.uniform(R_MIN, R_MAX)
    n_primary = int(DENSITY_FACTOR * (R / SCALE) ** 2)

    # uniform in disc
    r     = R * np.sqrt(rng_cls.random(n_primary))
    theta = rng_cls.random(n_primary) * 2 * np.pi
    xs0   = cx_i + r * np.cos(theta)
    ys0   = cy_i + r * np.sin(theta)

    keep = in_domain[np.clip(ys0.astype(int), 0, Ny - 1),
                     np.clip(xs0.astype(int), 0, Nx - 1)]
    primary = np.column_stack([xs0[keep], ys0[keep]])
    pts.extend(primary)

    if MULTIPLIER_LOCAL:
        jitter   = rng_mul.random((len(primary) * MULTIPLIER_LOCAL, 2)) - 0.5
        expanded = np.repeat(primary, MULTIPLIER_LOCAL, axis=0) + jitter
        keep2 = in_domain[np.clip(expanded[:, 1].astype(int), 0, Ny - 1),
                          np.clip(expanded[:, 0].astype(int), 0, Nx - 1)]
        pts.extend(expanded[keep2])

pts = np.asarray(pts, float)
pts = enforce_occupancy(pts, pts)  # ensure initial occupancy within limits
rng_step = np.random.default_rng(0)


# Velocity fields
# Phase 1 (ventral flow)
vx1 = np.zeros((Ny, Nx))
vy1 = np.zeros_like(vx1)
vy1[:] = -0.123 * SCALE   #-0.123 is derived experimentally from control samples

# Phase 2 (chiral flow)
vx2 = np.zeros_like(vx1)
vy2 = np.zeros_like(vx1)

# x-component: opposite signs on left/right halves (chiral directionality)
vx2[:Nx//3+3] =  +0.087 * SCALE #0.087 is derived experimentally from control samples
vx2[Nx//3+3:] = -0.052 * SCALE #-0.052 is derived experimentally from control samples

# y-component: stronger ventral component in the bottom third than in the dorsal two-thirds
vy2[Nx//3+3:] = -0.052 * SCALE     # dorsal (top 2/3), #-0.052 is derived experimentally from control samples
vy2[:Nx//3+3] =  -0.034 * SCALE     # ventral (bottom 1/3), #-0.034 is derived experimentally from control samples

def vel_at(xf, yf, vx, vy):
    """Sample the phase-specific velocity field at (xf, yf) using nearest-neighbour lookup."""
    xi = np.clip(np.rint(xf).astype(int), 0, Nx - 1)
    yi = np.clip(np.rint(yf).astype(int), 0, Ny - 1)
    return vx[yi, xi], vy[yi, xi]


def step(k, P):
    """
    Advance one Euler–Maruyama step with an affine correction:

        p_i ← p_i + dt [ V_c + J (p_i − p̄) ] + sqrt(2 D_lat dt) ξ_i

    where V_c is the mean sampled velocity, and J is a 2×2 velocity-gradient tensor
    fitted by least squares to (relative velocities) vs (relative positions).
    """
    vx, vy = (vx1, vy1) if k < PHASE1_STEPS else (vx2, vy2)
    vx_p, vy_p = vel_at(P[:, 0], P[:, 1], vx, vy)
    V = np.column_stack((vx_p, vy_p))

    # Affine fit about the centroid
    Pc = P.mean(0)    # centroid
    Vc = V.mean(0)    # mean velocity
    R  = P - Pc
    Vr = V - Vc
    RTR = R.T @ R
    J = (Vr.T @ R) @ np.linalg.inv(RTR) if np.linalg.cond(RTR) < 1e12 else np.zeros((2, 2))

    # Euler–Maruyama update
    Pn = P + dt * (Vc + (J @ R.T).T) \
           + np.sqrt(2 * D_LAT * dt) * rng_step.standard_normal(P.shape)

    # Boundary & occupancy constraints
    Pn = reflect_capsule(P, Pn)
    Pn = enforce_occupancy(P, Pn)
    return Pn


def raster(P):
    """Rasterize particle counts m[y, x] with origin at lower left (for imshow(origin='lower'))."""
    m = np.zeros((Ny, Nx))
    xi = np.clip(np.rint(P[:, 0]).astype(int), 0, Nx - 1)
    yi = np.clip(np.rint(P[:, 1]).astype(int), 0, Ny - 1)
    np.add.at(m, (yi, xi), 1)
    return m


# Visualization set-up 
fig = plt.figure(figsize=(10, 4))
gs  = fig.add_gridspec(1, 3, width_ratios=[1.2, 1.2, 1.2])
ax0, ax1, ax2 = [fig.add_subplot(gs[i]) for i in range(3)]

m0   = raster(pts)
nz0  = m0[m0 > 0]
vmax0 = np.percentile(nz0, PCLIP) if nz0.size else 1

im0 = ax0.imshow(m0, origin='lower', cmap='inferno', vmin=1, vmax=vmax0)
ax0.set_title('simulated cadherin localization'); ax0.axis('off')

# Column profiles at two x positions (used to measure cadherin signal intensity profile in Fig. 4E)
sel_offsets = np.array([-7, 7])                 # unscaled offsets from center (pixels/SCALE in µm)
sel_x_px    = cx + sel_offsets * SCALE
REF_SHIFT   = 7
y_coords    = (np.arange(Ny) - cy) / SCALE + REF_SHIFT

lines = [ax1.plot([], [], label=f'x={off:+d}')[0] for off in sel_offsets]
ax1.set_title('HMR-1 signal near the left or right surface (y = 0: cleavage furrow)')
ax1.set_xlabel('normalized occupancy (Σ_y = 1)')
ax1.set_ylabel('y  (0 ≡ cy – 7 px)')
ax1.invert_yaxis(); ax1.legend()
ax1.axhline(0, color='black', ls='--', lw=1)

# Capsule outline overlay (pixel-accurate; 0.5 shift aligns edges to pixel centers)
for ax in (ax0, ax2):
    patch = FancyBboxPatch((cx_dom - CAPS_R + 0.5, cy_dom - CAPS_HALF_L + 0.5),
                           2 * CAPS_R, 2 * CAPS_HALF_L,
                           boxstyle=f"round,pad=0,rounding_size={CAPS_R}",
                           edgecolor='white', facecolor='none', lw=1.5, ls=':')
    ax.add_patch(patch); ax.set_aspect('equal')

# Top-intensity panel
im2base = ax2.imshow(m0, origin='lower', cmap='inferno', vmin=1, vmax=vmax0)
if TOP_PERC is not None:
    im2 = ax2.imshow(np.where(mask_caps, 0, np.nan), origin='lower',
                     cmap='Reds', vmin=0, vmax=1, alpha=0.6)
else:
    im2 = ax2.imshow(np.full_like(m0, np.nan, dtype=float), origin='lower',
                     cmap='Reds', vmin=0, vmax=1, alpha=0.6)

ax2.set_title('vlines: positions used for intensity profile'); ax2.axis('off')
ax2.axvline(cx - 7 * SCALE, color='white', ls='--', lw=1)
ax2.axvline(cx + 7 * SCALE, color='white', ls='--', lw=1)
ax2.axhline(cy - 7 * SCALE, color='white', ls='--', lw=1)

# Velocity quiver (updates by phase)
skip = 4 * SCALE
xq, yq = np.arange(0, Nx, skip), np.arange(0, Ny, skip)
Xq, Yq = np.meshgrid(xq, yq)
quiv = ax0.quiver(Xq, Yq, vx1[Yq, Xq], vy1[Yq, Xq], scale=5, headwidth=6, headlength=8)

fig.tight_layout()
title_text = fig.suptitle('')


def update(frame):
    """Animation callback: integrate SAVE_EVERY steps, then refresh rasters/overlays."""
    global pts
    k = frame * SAVE_EVERY

    for _ in range(SAVE_EVERY):
        pts = step(k, pts)

    m = raster(pts)
    nz = m[m > 0]
    vmax = np.percentile(nz, PCLIP) if nz.size else 1

    # update images
    im0.set_data(m);      im0.set_clim(1, vmax)
    im2base.set_data(m);  im2base.set_clim(1, vmax)

    # update quiver according to phase
    vx, vy = (vx1, vy1) if k < PHASE1_STEPS else (vx2, vy2)
    quiv.set_UVC(vx[Yq, Xq], vy[Yq, Xq])

    # update column profiles (normalize each column to Σ_y = 1)
    for l, x in zip(lines, sel_x_px):
        col = m[:, x].astype(float)
        total = col.sum()
        l.set_data(col / total if total else col, y_coords)

    ax1.set_xlim(0, 1)
    ax1.set_ylim(y_coords.min(), y_coords.max())

    # top-intensity overlay within the capsule
    if TOP_PERC is not None:
        thresh = np.percentile(m[mask_caps], TOP_PERC)
        im2.set_data(np.where(mask_caps & (m >= thresh), 1, np.nan))
    else:
        im2.set_data(np.full_like(m, np.nan, dtype=float))

    title_text.set_text(f"t = {k*dt:.1f} s   ({'phase-1' if k < PHASE1_STEPS else 'phase-2'})")
    return [im0, im2base, quiv, im2, *lines, title_text]


ani = FuncAnimation(fig, update,
                    frames=np.arange(1, STEPS // SAVE_EVERY + 1),
                    blit=True, interval=1000*dt)
HTML(ani.to_jshtml())

# Cadherin distribution simulation (no ventral flow; devitellinized embryos)

In [12]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import FancyBboxPatch
from IPython.display import HTML

plt.rcParams['animation.embed_limit'] = 200

# Parameters
SCALE             = 2    # dimensionless scale factor (pixels per µm)
Nx = Ny           = 60 * SCALE   # raster size (pixels)
dt                = 0.1          # time step (s)
STEPS             = 1310         # total integration steps for control sample
SAVE_EVERY        = 2            # render every N steps
MAX_PER_PIXEL     = 14           # occupancy cap per pixel (particles)

# Diffusion: literature value for E-cadherin (Erami et al., 2016), in µm²/s.
D_LAT_UM2_S       = 0.0048       # µm²/s
D_LAT             = D_LAT_UM2_S * (SCALE ** 2)   # pixel²/s

# Two-phase advection visualization controls
PHASE1_STEPS      = 630          # end of phase 1 (ventral flow, control)
PCLIP             = 90           # percentile for colour stretch (each frame)
TOP_PERC          = 95           # overlay ≥TOP_PERC percentile if not None

# Cell-cell contact area definition
cx_dom = cy_dom = 30 * SCALE

# 2D capsule geometry
CAPS_R       = 13 * SCALE
CAPS_HALF_L  = 16 * SCALE

# Inner corner points along the capsule's center line (used for distance queries)
A = np.array([cx_dom, cy_dom - (CAPS_HALF_L - CAPS_R)])  # top inner corner
B = np.array([cx_dom, cy_dom + (CAPS_HALF_L - CAPS_R)])  # bottom inner corner
BA      = B - A
BA_dot  = BA @ BA

# Functions
def signed_dist_capsule(P):
    """Return +distance inside, −outside for a vertical capsule."""
    pa = P - A
    h  = np.clip((pa @ BA) / BA_dot, 0.0, 1.0)[:, None]
    closest = A + h * BA
    return CAPS_R - np.linalg.norm(P - closest, axis=1)  # >0 inside

def reflect_capsule(P_old, P_new):
    """Specular reflection at the capsule boundary."""
    out = signed_dist_capsule(P_new) < 0
    if not np.any(out):
        return P_new
    P_out = P_new[out]
    pa    = P_out - A
    h     = np.clip((pa @ BA) / BA_dot, 0.0, 1.0)[:, None]
    nearest = A + h * BA
    n_vec   = P_out - nearest
    n_hat   = n_vec / np.linalg.norm(n_vec, axis=1, keepdims=True)
    d   = P_out - P_old[out]
    d_n = (d * n_hat).sum(axis=1, keepdims=True) * n_hat
    P_new[out] = P_old[out] + d - 2 * d_n
    return P_new

def enforce_occupancy(P_old, P_new, max_occ=MAX_PER_PIXEL):
    """Reject moves into pixels whose occupancy would exceed max_occ."""
    ix = np.clip(np.rint(P_new[:, 0]).astype(int), 0, Nx - 1)
    iy = np.clip(np.rint(P_new[:, 1]).astype(int), 0, Ny - 1)
    flat = iy * Nx + ix
    uniq, cnts = np.unique(flat, return_counts=True)
    crowded = uniq[cnts > max_occ]
    if crowded.size:
        mask = np.isin(flat, crowded)
        P_new[mask] = P_old[mask]
    return P_new

# Initial condition: clustered cadherin patch with holes
cx = cy = Nx // 2
PATCH_RAD = 8 * SCALE
Y, X = np.indices((Ny, Nx))
circ = (X - cx) ** 2 + (Y - cy) ** 2 <= PATCH_RAD ** 2

# Capsule mask
mask_caps = signed_dist_capsule(np.column_stack([X.ravel(), Y.ravel()])).reshape(Ny, Nx) > 0

# Three small “holes”
rng_hole = np.random.default_rng(27)
hole = np.zeros_like(circ, bool)
for _ in range(3):
    while True:
        hx = rng_hole.integers(cx - PATCH_RAD, cx + PATCH_RAD + 1)
        hy = rng_hole.integers(cy - PATCH_RAD, cy + PATCH_RAD + 1)
        if (hx - cx) ** 2 + (hy - cy) ** 2 <= PATCH_RAD ** 2:
            break
    hole |= (X - hx) ** 2 + (Y - hy) ** 2 <= (3 ** 2)

in_domain = circ & mask_caps & ~hole

# Random-size clusters (primary samples + local jitter)
N_CLUSTERS       = 20
R_MIN, R_MAX     = 2 * SCALE, 5 * SCALE
DENSITY_FACTOR   = 8
MULTIPLIER_LOCAL = 1
rng_cls  = np.random.default_rng(32)
rng_mul  = np.random.default_rng(32)

pts = []
for _ in range(N_CLUSTERS):
    while True:
        cx_i = rng_cls.integers(cx - PATCH_RAD, cx + PATCH_RAD + 1)
        cy_i = rng_cls.integers(cy - PATCH_RAD, cy + PATCH_RAD + 1)
        if (cx_i - cx) ** 2 + (cy_i - cy) ** 2 <= PATCH_RAD ** 2:
            break
    R = rng_cls.uniform(R_MIN, R_MAX)
    n_primary = int(DENSITY_FACTOR * (R / SCALE) ** 2)
    r     = R * np.sqrt(rng_cls.random(n_primary))
    theta = rng_cls.random(n_primary) * 2 * np.pi
    xs0   = cx_i + r * np.cos(theta)
    ys0   = cy_i + r * np.sin(theta)
    keep = in_domain[np.clip(ys0.astype(int), 0, Ny - 1),
                     np.clip(xs0.astype(int), 0, Nx - 1)]
    primary = np.column_stack([xs0[keep], ys0[keep]])
    pts.extend(primary)

    if MULTIPLIER_LOCAL:
        jitter   = rng_mul.random((len(primary) * MULTIPLIER_LOCAL, 2)) - 0.5
        expanded = np.repeat(primary, MULTIPLIER_LOCAL, axis=0) + jitter
        keep2 = in_domain[np.clip(expanded[:, 1].astype(int), 0, Ny - 1),
                          np.clip(expanded[:, 0].astype(int), 0, Nx - 1)]
        pts.extend(expanded[keep2])

pts = np.asarray(pts, float)
pts = enforce_occupancy(pts, pts)
rng_step = np.random.default_rng(0)

# Velocity fields
# Phase 1 (ventral flow)
vx1 = np.zeros((Ny, Nx))
vy1 = np.zeros_like(vx1)
vy1[:] = 0 * SCALE

# Phase 2 (chiral flow) — NOTE: this uses Nx on y-slices as in your original;
# if you meant dorsal/ventral splits, change Nx → Ny on vy2 slices.
vx2 = np.zeros_like(vx1)
vy2 = np.zeros_like(vx1)

# x-component: opposite signs on left/right halves
vx2[:Nx//2] =  +0.087 * SCALE
vx2[Nx//2:] = -0.052 * SCALE

# y-component (kept verbatim from your code; see note above)
vy2[Nx//2:] = 0 * SCALE
vy2[:Nx//2] = 0 * SCALE

def vel_at(xf, yf, vx, vy):
    xi = np.clip(np.rint(xf).astype(int), 0, Nx - 1)
    yi = np.clip(np.rint(yf).astype(int), 0, Ny - 1)
    return vx[yi, xi], vy[yi, xi]

def step(k, P):
    """Euler–Maruyama step with affine correction."""
    vx, vy = (vx1, vy1) if k < PHASE1_STEPS else (vx2, vy2)
    vx_p, vy_p = vel_at(P[:, 0], P[:, 1], vx, vy)
    V = np.column_stack((vx_p, vy_p))

    Pc = P.mean(0);  Vc = V.mean(0)
    R  = P - Pc;     Vr = V - Vc
    RTR = R.T @ R
    J = (Vr.T @ R) @ np.linalg.inv(RTR) if np.linalg.cond(RTR) < 1e12 else np.zeros((2, 2))

    Pn = P + dt * (Vc + (J @ R.T).T) \
           + np.sqrt(2 * D_LAT * dt) * rng_step.standard_normal(P.shape)

    Pn = reflect_capsule(P, Pn)
    Pn = enforce_occupancy(P, Pn)
    return Pn

def raster(P):
    """Rasterize particle counts m[y, x] (origin='lower')."""
    m = np.zeros((Ny, Nx))
    xi = np.clip(np.rint(P[:, 0]).astype(int), 0, Nx - 1)
    yi = np.clip(np.rint(P[:, 1]).astype(int), 0, Ny - 1)
    np.add.at(m, (yi, xi), 1)
    return m

# Visualization set-up
fig = plt.figure(figsize=(10, 4))
gs  = fig.add_gridspec(1, 3, width_ratios=[1.2, 1.2, 1.2])
ax0, ax1, ax2 = [fig.add_subplot(gs[i]) for i in range(3)]

m0   = raster(pts)
nz0  = m0[m0 > 0]
vmax0 = np.percentile(nz0, PCLIP) if nz0.size else 1

im0 = ax0.imshow(m0, origin='lower', cmap='inferno', vmin=1, vmax=vmax0)
ax0.set_title('simulated cadherin localization'); ax0.axis('off')

sel_offsets = np.array([-7, 7])
sel_x_px    = cx + sel_offsets * SCALE
REF_SHIFT   = 0
y_coords    = (np.arange(Ny) - cy) / SCALE + REF_SHIFT

lines = [ax1.plot([], [], label=f'x={off:+d}')[0] for off in sel_offsets]
ax1.set_title('HMR-1 signal near the left or right surface (y = 0: cleavage furrow)')
ax1.set_xlabel('normalized occupancy (Σ_y = 1)')
ax1.set_ylabel('y  (0 ≡ cy – 0 px)')
ax1.invert_yaxis(); ax1.legend()
ax1.axhline(0, color='black', ls='--', lw=1)

for ax in (ax0, ax2):
    patch = FancyBboxPatch((cx_dom - CAPS_R + 0.5, cy_dom - CAPS_HALF_L + 0.5),
                           2 * CAPS_R, 2 * CAPS_HALF_L,
                           boxstyle=f"round,pad=0,rounding_size={CAPS_R}",
                           edgecolor='white', facecolor='none', lw=1.5, ls=':')
    ax.add_patch(patch); ax.set_aspect('equal')

im2base = ax2.imshow(m0, origin='lower', cmap='inferno', vmin=1, vmax=vmax0)
if TOP_PERC is not None:
    im2 = ax2.imshow(np.where(mask_caps, 0, np.nan), origin='lower',
                     cmap='Reds', vmin=0, vmax=1, alpha=0.6)
else:
    im2 = ax2.imshow(np.full_like(m0, np.nan, dtype=float), origin='lower',
                     cmap='Reds', vmin=0, vmax=1, alpha=0.6)

ax2.set_title('vlines: positions used for intensity profile'); ax2.axis('off')
ax2.axvline(cx - 7 * SCALE, color='white', ls='--', lw=1)
ax2.axvline(cx + 7 * SCALE, color='white', ls='--', lw=1)
ax2.axhline(cy*SCALE, color='white', ls='--', lw=1)

skip = 4 * SCALE
xq, yq = np.arange(0, Nx, skip), np.arange(0, Ny, skip)
Xq, Yq = np.meshgrid(xq, yq)
quiv = ax0.quiver(Xq, Yq, vx1[Yq, Xq], vy1[Yq, Xq], scale=5, headwidth=6, headlength=8, color = (1,1,1,0.6), linewidth =0.0, zorder =5)

fig.tight_layout()
title_text = fig.suptitle('')

def update(frame):
    """Integrate SAVE_EVERY steps, then refresh rasters/overlays."""
    global pts
    k = frame * SAVE_EVERY
    for _ in range(SAVE_EVERY):
        pts = step(k, pts)

    m = raster(pts)
    nz = m[m > 0]
    vmax = np.percentile(nz, PCLIP) if nz.size else 1

    im0.set_data(m);      im0.set_clim(1, vmax)
    im2base.set_data(m);  im2base.set_clim(1, vmax)

    vx, vy = (vx1, vy1) if k < PHASE1_STEPS else (vx2, vy2)
    quiv.set_UVC(vx[Yq, Xq], vy[Yq, Xq])

    for l, x in zip(lines, sel_x_px):
        col = m[:, x].astype(float)
        total = col.sum()
        l.set_data(col / total if total else col, y_coords)

    ax1.set_xlim(0, 1)
    ax1.set_ylim(y_coords.min(), y_coords.max())

    if TOP_PERC is not None:
        thresh = np.percentile(m[mask_caps], TOP_PERC)
        im2.set_data(np.where(mask_caps & (m >= thresh), 1, np.nan))
    else:
        im2.set_data(np.full_like(m, np.nan, dtype=float))

    title_text.set_text(f"t = {k*dt:.1f} s   ({'phase-1' if k < PHASE1_STEPS else 'phase-2'})")
    return [im0, im2base, quiv, im2, *lines, title_text]

# Build animation
frames = np.arange(0, STEPS // SAVE_EVERY + 1)
ani = FuncAnimation(fig, update, frames=frames, blit=True, interval=1000*dt)


# -------- MP4 export --------
fps = 30
mp4_path = 'cadherin_simulation_noventral.mp4'
try:
    from matplotlib.animation import FFMpegWriter
    writer = FFMpegWriter(
        fps=fps,
        codec='libx264',
        bitrate=3000,
        extra_args=['-pix_fmt', 'yuv420p']  # broad compatibility (PowerPoint, QuickTime)
    )
    ani.save(mp4_path, writer=writer, dpi=200)
    print(f"✓ Saved MP4 to {mp4_path}  (fps={fps}, dpi=200)")
except Exception as e:
    # fallback: try string writer if FFMpegWriter import succeeded but codec/path not found
    try:
        ani.save(mp4_path, writer='ffmpeg', fps=fps, dpi=200, extra_args=['-pix_fmt', 'yuv420p'])
        print(f"✓ Saved MP4 to {mp4_path}  (fps={fps}, dpi=200)")
    except Exception as ee:
        print("✗ Could not write MP4. Install ffmpeg (e.g., `conda install -c conda-forge ffmpeg`) and retry.")
        print("  Error:", ee)


✓ Saved MP4 to cadherin_simulation_noventral.mp4  (fps=30, dpi=200)


# Cadherin distribution simulation (ect-2(8hr))

In [13]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import FancyBboxPatch
from IPython.display import HTML

plt.rcParams['animation.embed_limit'] = 200

# Parameters
SCALE             = 2    # dimensionless scale factor (pixels per µm)
Nx = Ny           = 60 * SCALE   # raster size (pixels)
dt                = 0.1          # time step (s)
STEPS             = 1412         # total integration steps for control sample
SAVE_EVERY        = 2            # render every N steps
MAX_PER_PIXEL     = 14           # occupancy cap per pixel (particles)

# Diffusion: literature value for E-cadherin (Erami et al., 2016), in µm²/s.
D_LAT_UM2_S       = 0.0048       # µm²/s
D_LAT             = D_LAT_UM2_S * (SCALE ** 2)   # pixel²/s

# Two-phase advection visualization controls
PHASE1_STEPS      = 871          # end of phase 1 (ventral flow, control)
PCLIP             = 90           # percentile for colour stretch (each frame)
TOP_PERC          = 95           # overlay ≥TOP_PERC percentile if not None

# Cell-cell contact area definition
cx_dom = cy_dom = 30 * SCALE

# 2D capsule geometry
CAPS_R       = 13 * SCALE
CAPS_HALF_L  = 16 * SCALE

# Inner corner points along the capsule's center line (used for distance queries)
A = np.array([cx_dom, cy_dom - (CAPS_HALF_L - CAPS_R)])  # top inner corner
B = np.array([cx_dom, cy_dom + (CAPS_HALF_L - CAPS_R)])  # bottom inner corner
BA      = B - A
BA_dot  = BA @ BA

# Functions
def signed_dist_capsule(P):
    """Return +distance inside, −outside for a vertical capsule."""
    pa = P - A
    h  = np.clip((pa @ BA) / BA_dot, 0.0, 1.0)[:, None]
    closest = A + h * BA
    return CAPS_R - np.linalg.norm(P - closest, axis=1)  # >0 inside

def reflect_capsule(P_old, P_new):
    """Specular reflection at the capsule boundary."""
    out = signed_dist_capsule(P_new) < 0
    if not np.any(out):
        return P_new
    P_out = P_new[out]
    pa    = P_out - A
    h     = np.clip((pa @ BA) / BA_dot, 0.0, 1.0)[:, None]
    nearest = A + h * BA
    n_vec   = P_out - nearest
    n_hat   = n_vec / np.linalg.norm(n_vec, axis=1, keepdims=True)
    d   = P_out - P_old[out]
    d_n = (d * n_hat).sum(axis=1, keepdims=True) * n_hat
    P_new[out] = P_old[out] + d - 2 * d_n
    return P_new

def enforce_occupancy(P_old, P_new, max_occ=MAX_PER_PIXEL):
    """Reject moves into pixels whose occupancy would exceed max_occ."""
    ix = np.clip(np.rint(P_new[:, 0]).astype(int), 0, Nx - 1)
    iy = np.clip(np.rint(P_new[:, 1]).astype(int), 0, Ny - 1)
    flat = iy * Nx + ix
    uniq, cnts = np.unique(flat, return_counts=True)
    crowded = uniq[cnts > max_occ]
    if crowded.size:
        mask = np.isin(flat, crowded)
        P_new[mask] = P_old[mask]
    return P_new

# Initial condition: clustered cadherin patch with holes
cx = cy = Nx // 2
PATCH_RAD = 8 * SCALE
Y, X = np.indices((Ny, Nx))
circ = (X - cx) ** 2 + (Y - cy) ** 2 <= PATCH_RAD ** 2

# Capsule mask
mask_caps = signed_dist_capsule(np.column_stack([X.ravel(), Y.ravel()])).reshape(Ny, Nx) > 0

# Three small “holes”
rng_hole = np.random.default_rng(27)
hole = np.zeros_like(circ, bool)
for _ in range(3):
    while True:
        hx = rng_hole.integers(cx - PATCH_RAD, cx + PATCH_RAD + 1)
        hy = rng_hole.integers(cy - PATCH_RAD, cy + PATCH_RAD + 1)
        if (hx - cx) ** 2 + (hy - cy) ** 2 <= PATCH_RAD ** 2:
            break
    hole |= (X - hx) ** 2 + (Y - hy) ** 2 <= (3 ** 2)

in_domain = circ & mask_caps & ~hole

# Random-size clusters (primary samples + local jitter)
N_CLUSTERS       = 20
R_MIN, R_MAX     = 2 * SCALE, 5 * SCALE
DENSITY_FACTOR   = 8
MULTIPLIER_LOCAL = 1
rng_cls  = np.random.default_rng(32)
rng_mul  = np.random.default_rng(32)

pts = []
for _ in range(N_CLUSTERS):
    while True:
        cx_i = rng_cls.integers(cx - PATCH_RAD, cx + PATCH_RAD + 1)
        cy_i = rng_cls.integers(cy - PATCH_RAD, cy + PATCH_RAD + 1)
        if (cx_i - cx) ** 2 + (cy_i - cy) ** 2 <= PATCH_RAD ** 2:
            break
    R = rng_cls.uniform(R_MIN, R_MAX)
    n_primary = int(DENSITY_FACTOR * (R / SCALE) ** 2)
    r     = R * np.sqrt(rng_cls.random(n_primary))
    theta = rng_cls.random(n_primary) * 2 * np.pi
    xs0   = cx_i + r * np.cos(theta)
    ys0   = cy_i + r * np.sin(theta)
    keep = in_domain[np.clip(ys0.astype(int), 0, Ny - 1),
                     np.clip(xs0.astype(int), 0, Nx - 1)]
    primary = np.column_stack([xs0[keep], ys0[keep]])
    pts.extend(primary)

    if MULTIPLIER_LOCAL:
        jitter   = rng_mul.random((len(primary) * MULTIPLIER_LOCAL, 2)) - 0.5
        expanded = np.repeat(primary, MULTIPLIER_LOCAL, axis=0) + jitter
        keep2 = in_domain[np.clip(expanded[:, 1].astype(int), 0, Ny - 1),
                          np.clip(expanded[:, 0].astype(int), 0, Nx - 1)]
        pts.extend(expanded[keep2])

pts = np.asarray(pts, float)
pts = enforce_occupancy(pts, pts)
rng_step = np.random.default_rng(0)

# Velocity fields
# Phase 1 (ventral flow)
vx1 = np.zeros((Ny, Nx))
vy1 = np.zeros_like(vx1)
vy1[:] = -0.0580 * SCALE

# Phase 2 (chiral flow) — NOTE: this uses Nx on y-slices as in your original;
# if you meant dorsal/ventral splits, change Nx → Ny on vy2 slices.
vx2 = np.zeros_like(vx1)
vy2 = np.zeros_like(vx1)

# x-component: opposite signs on left/right halves
vx2[:Nx//3+3] =  -0.0046 * SCALE
vx2[Nx//3+3:] = -0.00557 * SCALE

# y-component (kept verbatim from your code; see note above)
vy2[Nx//3+3:] = -0.0247 * SCALE
vy2[:Nx//3+3] = -0.0054 * SCALE

def vel_at(xf, yf, vx, vy):
    xi = np.clip(np.rint(xf).astype(int), 0, Nx - 1)
    yi = np.clip(np.rint(yf).astype(int), 0, Ny - 1)
    return vx[yi, xi], vy[yi, xi]

def step(k, P):
    """Euler–Maruyama step with affine correction."""
    vx, vy = (vx1, vy1) if k < PHASE1_STEPS else (vx2, vy2)
    vx_p, vy_p = vel_at(P[:, 0], P[:, 1], vx, vy)
    V = np.column_stack((vx_p, vy_p))

    Pc = P.mean(0);  Vc = V.mean(0)
    R  = P - Pc;     Vr = V - Vc
    RTR = R.T @ R
    J = (Vr.T @ R) @ np.linalg.inv(RTR) if np.linalg.cond(RTR) < 1e12 else np.zeros((2, 2))

    Pn = P + dt * (Vc + (J @ R.T).T) \
           + np.sqrt(2 * D_LAT * dt) * rng_step.standard_normal(P.shape)

    Pn = reflect_capsule(P, Pn)
    Pn = enforce_occupancy(P, Pn)
    return Pn

def raster(P):
    """Rasterize particle counts m[y, x] (origin='lower')."""
    m = np.zeros((Ny, Nx))
    xi = np.clip(np.rint(P[:, 0]).astype(int), 0, Nx - 1)
    yi = np.clip(np.rint(P[:, 1]).astype(int), 0, Ny - 1)
    np.add.at(m, (yi, xi), 1)
    return m

# Visualization set-up
fig = plt.figure(figsize=(10, 4))
gs  = fig.add_gridspec(1, 3, width_ratios=[1.2, 1.2, 1.2])
ax0, ax1, ax2 = [fig.add_subplot(gs[i]) for i in range(3)]

m0   = raster(pts)
nz0  = m0[m0 > 0]
vmax0 = np.percentile(nz0, PCLIP) if nz0.size else 1

im0 = ax0.imshow(m0, origin='lower', cmap='inferno', vmin=1, vmax=vmax0)
ax0.set_title('simulated cadherin localization'); ax0.axis('off')

sel_offsets = np.array([-7, 7])
sel_x_px    = cx + sel_offsets * SCALE
REF_SHIFT   = 7
y_coords    = (np.arange(Ny) - cy) / SCALE + REF_SHIFT

lines = [ax1.plot([], [], label=f'x={off:+d}')[0] for off in sel_offsets]
ax1.set_title('HMR-1 signal near the left or right surface (y = 0: cleavage furrow)')
ax1.set_xlabel('normalized occupancy (Σ_y = 1)')
ax1.set_ylabel('y  (0 ≡ cy – 7 px)')
ax1.invert_yaxis(); ax1.legend()
ax1.axhline(0, color='black', ls='--', lw=1)

for ax in (ax0, ax2):
    patch = FancyBboxPatch((cx_dom - CAPS_R + 0.5, cy_dom - CAPS_HALF_L + 0.5),
                           2 * CAPS_R, 2 * CAPS_HALF_L,
                           boxstyle=f"round,pad=0,rounding_size={CAPS_R}",
                           edgecolor='white', facecolor='none', lw=1.5, ls=':')
    ax.add_patch(patch); ax.set_aspect('equal')

im2base = ax2.imshow(m0, origin='lower', cmap='inferno', vmin=1, vmax=vmax0)
if TOP_PERC is not None:
    im2 = ax2.imshow(np.where(mask_caps, 0, np.nan), origin='lower',
                     cmap='Reds', vmin=0, vmax=1, alpha=0.6)
else:
    im2 = ax2.imshow(np.full_like(m0, np.nan, dtype=float), origin='lower',
                     cmap='Reds', vmin=0, vmax=1, alpha=0.6)

ax2.set_title('vlines: positions used for intensity profile'); ax2.axis('off')
ax2.axvline(cx - 7 * SCALE, color='white', ls='--', lw=1)
ax2.axvline(cx + 7 * SCALE, color='white', ls='--', lw=1)
ax2.axhline(cy - 7 * SCALE, color='white', ls='--', lw=1)

skip = 4 * SCALE
xq, yq = np.arange(0, Nx, skip), np.arange(0, Ny, skip)
Xq, Yq = np.meshgrid(xq, yq)
quiv = ax0.quiver(Xq, Yq, vx1[Yq, Xq], vy1[Yq, Xq], scale=5, headwidth=6, headlength=8, color = (1,1,1,0.6), linewidth =0.0, zorder =5)


fig.tight_layout()
title_text = fig.suptitle('')

def update(frame):
    """Integrate SAVE_EVERY steps, then refresh rasters/overlays."""
    global pts
    k = frame * SAVE_EVERY
    for _ in range(SAVE_EVERY):
        pts = step(k, pts)

    m = raster(pts)
    nz = m[m > 0]
    vmax = np.percentile(nz, PCLIP) if nz.size else 1

    im0.set_data(m);      im0.set_clim(1, vmax)
    im2base.set_data(m);  im2base.set_clim(1, vmax)

    vx, vy = (vx1, vy1) if k < PHASE1_STEPS else (vx2, vy2)
    quiv.set_UVC(vx[Yq, Xq], vy[Yq, Xq])

    for l, x in zip(lines, sel_x_px):
        col = m[:, x].astype(float)
        total = col.sum()
        l.set_data(col / total if total else col, y_coords)

    ax1.set_xlim(0, 1)
    ax1.set_ylim(y_coords.min(), y_coords.max())

    if TOP_PERC is not None:
        thresh = np.percentile(m[mask_caps], TOP_PERC)
        im2.set_data(np.where(mask_caps & (m >= thresh), 1, np.nan))
    else:
        im2.set_data(np.full_like(m, np.nan, dtype=float))

    title_text.set_text(f"t = {k*dt:.1f} s   ({'phase-1' if k < PHASE1_STEPS else 'phase-2'})")
    return [im0, im2base, quiv, im2, *lines, title_text]

# Build animation
frames = np.arange(0, STEPS // SAVE_EVERY + 1)
ani = FuncAnimation(fig, update, frames=frames, blit=True, interval=1000*dt)


# -------- MP4 export --------
fps = 30
mp4_path = 'cadherin_simulation_ect2.mp4'
try:
    from matplotlib.animation import FFMpegWriter
    writer = FFMpegWriter(
        fps=fps,
        codec='libx264',
        bitrate=3000,
        extra_args=['-pix_fmt', 'yuv420p']  # broad compatibility (PowerPoint, QuickTime)
    )
    ani.save(mp4_path, writer=writer, dpi=200)
    print(f"✓ Saved MP4 to {mp4_path}  (fps={fps}, dpi=200)")
except Exception as e:
    # fallback: try string writer if FFMpegWriter import succeeded but codec/path not found
    try:
        ani.save(mp4_path, writer='ffmpeg', fps=fps, dpi=200, extra_args=['-pix_fmt', 'yuv420p'])
        print(f"✓ Saved MP4 to {mp4_path}  (fps={fps}, dpi=200)")
    except Exception as ee:
        print("✗ Could not write MP4. Install ffmpeg (e.g., `conda install -c conda-forge ffmpeg`) and retry.")
        print("  Error:", ee)


✓ Saved MP4 to cadherin_simulation_ect2.mp4  (fps=30, dpi=200)


# Monte-Carlo simulation to generate Figure 4E intensity profiles

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# progress bar (widget when available, text otherwise)
try:
    from tqdm.notebook import tqdm
    _ = tqdm(range(1), leave=False); _.close()
except Exception:
    from tqdm import tqdm
    print("tqdm widget unavailable – using text-mode progress bar.")

# ---------------- Parameters ----------------
SCALE             = 2            # pixels per µm (dimensionless scale factor)
Nx = Ny           = 60 * SCALE   # raster size (pixels)
dt                = 0.1          # time step (s)
STEPS             = 1310         # total steps (control)
SAVE_EVERY        = 2            # not used here, kept for consistency
MAX_PER_PIXEL     = 14           # occupancy cap
PHASE1_STEPS      = 630          # phase-1 (ventral) duration

# Diffusion (µm^2/s)
D_LAT_UM2_S       = 0.0048
D_LAT             = D_LAT_UM2_S * (SCALE**2)

# Cell-cell contact area definition
# Position of the cell-cell contact area
cx_dom = cy_dom = 30 * SCALE

# 2D capsule geometry (CAPS_R: radius of half circle, CAPS_HALF_L: half height of the total area)
CAPS_R       = 13 * SCALE
CAPS_HALF_L  = 16 * SCALE

# Inner corner points along the capsule's center line (used for distance queries)
A = np.array([cx_dom, cy_dom - (CAPS_HALF_L - CAPS_R)])  # top inner corner
B = np.array([cx_dom, cy_dom + (CAPS_HALF_L - CAPS_R)])  # bottom inner corner
BA      = B - A
BA_dot  = BA @ BA

# Functions
def signed_dist_capsule(P):
    """Positive inside, negative outside (vertical capsule)."""
    pa = P - A
    h  = np.clip((pa @ BA) / BA_dot, 0.0, 1.0)[:, None]
    closest = A + h * BA
    return CAPS_R - np.linalg.norm(P - closest, axis=1)

def reflect_capsule(P_old, P_new):
    """Specular reflection across the capsule boundary."""
    out = signed_dist_capsule(P_new) < 0
    if not np.any(out):
        return P_new
    P_out = P_new[out]
    pa    = P_out - A
    h     = np.clip((pa @ BA) / BA_dot, 0.0, 1.0)[:, None]
    nearest = A + h * BA
    n_vec   = P_out - nearest
    n_hat   = n_vec / np.linalg.norm(n_vec, axis=1, keepdims=True)
    d   = P_out - P_old[out]
    d_n = (d * n_hat).sum(axis=1, keepdims=True) * n_hat
    P_new[out] = P_old[out] + d - 2 * d_n
    return P_new

def enforce_occupancy(P_old, P_new, max_occ=MAX_PER_PIXEL):
    """Reject moves that would exceed per-pixel occupancy."""
    ix = np.clip(np.rint(P_new[:, 0]).astype(int), 0, Nx - 1)
    iy = np.clip(np.rint(P_new[:, 1]).astype(int), 0, Ny - 1)
    flat = iy * Nx + ix
    uniq, cnts = np.unique(flat, return_counts=True)
    crowded = uniq[cnts > max_occ]
    if crowded.size:
        mask = np.isin(flat, crowded)
        P_new[mask] = P_old[mask]
    return P_new

def raster(P):
    """Particle counts m[y, x] (origin lower-left for imshow(origin='lower'))."""
    m = np.zeros((Ny, Nx), dtype=int)
    xi = np.clip(np.rint(P[:, 0]).astype(int), 0, Nx - 1)
    yi = np.clip(np.rint(P[:, 1]).astype(int), 0, Ny - 1)
    np.add.at(m, (yi, xi), 1)
    return m

# Velocity fields
# Phase 1 (ventral flow)
vx1 = np.zeros((Ny, Nx))
vy1 = np.zeros_like(vx1)
vy1[:] = -0.123 * SCALE   #-0.123 is derived experimentally from control samples

# Phase 2 (chiral flow)
vx2 = np.zeros_like(vx1)
vy2 = np.zeros_like(vx1)

# x-component: opposite signs on left/right halves (chiral directionality)
vx2[:Nx//3+3] =  +0.087 * SCALE #0.087 is derived experimentally from control samples
vx2[Nx//3+3:] = -0.052 * SCALE #-0.052 is derived experimentally from control samples

# y-component: stronger ventral component in the bottom third than in the dorsal two-thirds
vy2[Nx//3+3:] = -0.052 * SCALE     # dorsal (top 2/3), #-0.052 is derived experimentally from control samples
vy2[:Nx//3+3] =  -0.034 * SCALE     # ventral (bottom 1/3), #-0.034 is derived experimentally from control samples

def vel_at(xf, yf, vx, vy):
    xi = np.clip(np.rint(xf).astype(int), 0, Nx - 1)
    yi = np.clip(np.rint(yf).astype(int), 0, Ny - 1)
    return vx[yi, xi], vy[yi, xi]

def one_step(k, P, rng_step):
    """Euler–Maruyama + affine advection around centroid."""
    vx, vy = (vx1, vy1) if k < PHASE1_STEPS else (vx2, vy2)
    vx_p, vy_p = vel_at(P[:, 0], P[:, 1], vx, vy)
    V = np.column_stack((vx_p, vy_p))

    Pc = P.mean(0);  Vc = V.mean(0)
    R  = P - Pc;     Vr = V - Vc
    RTR = R.T @ R
    J = (Vr.T @ R) @ np.linalg.inv(RTR) if np.linalg.cond(RTR) < 1e12 else np.zeros((2, 2))

    Pn = P + dt * (Vc + (J @ R.T).T) \
           + np.sqrt(2 * D_LAT * dt) * rng_step.standard_normal(P.shape)

    Pn = reflect_capsule(P, Pn)
    Pn = enforce_occupancy(P, Pn)
    return Pn

def run_one(seed):
    """Return normalised column profiles (Ny,) at x = cx ± 8*SCALE."""
    rng_cls  = np.random.default_rng(seed)
    rng_mul  = np.random.default_rng(seed + 1)
    rng_step = np.random.default_rng(seed + 2)

    # initial clustered patch with three 'holes'
    cx = cy = Nx // 2
    PATCH_RAD = 8 * SCALE
    Y, X = np.indices((Ny, Nx))
    circ = (X - cx) ** 2 + (Y - cy) ** 2 <= PATCH_RAD ** 2
    mask_caps = signed_dist_capsule(np.column_stack([X.ravel(), Y.ravel()])).reshape(Ny, Nx) > 0

    hole = np.zeros_like(circ, bool)
    for _ in range(3):
        while True:
            hx = rng_cls.integers(cx - PATCH_RAD, cx + PATCH_RAD + 1)
            hy = rng_cls.integers(cy - PATCH_RAD, cy + PATCH_RAD + 1)
            if (hx - cx) ** 2 + (hy - cy) ** 2 <= PATCH_RAD ** 2:
                break
        hole |= (X - hx) ** 2 + (Y - hy) ** 2 <= 3**2

    in_domain = circ & mask_caps & ~hole

    # clusters (primary + local jitter)
    pts = []
    R_MIN, R_MAX = 2 * SCALE, 5 * SCALE
    DENSITY_FACTOR, MULTIPLIER_LOCAL = 8, 1
    for _ in range(20):
        while True:
            cx_i = rng_cls.integers(cx - PATCH_RAD, cx + PATCH_RAD + 1)
            cy_i = rng_cls.integers(cy - PATCH_RAD, cy + PATCH_RAD + 1)
            if (cx_i - cx) ** 2 + (cy_i - cy) ** 2 <= PATCH_RAD ** 2:
                break
        R = rng_cls.uniform(R_MIN, R_MAX)
        n_primary = int(DENSITY_FACTOR * (R / SCALE) ** 2)
        r     = R * np.sqrt(rng_cls.random(n_primary))
        theta = rng_cls.random(n_primary) * 2 * np.pi
        xs0   = cx_i + r * np.cos(theta)
        ys0   = cy_i + r * np.sin(theta)

        keep = in_domain[np.clip(ys0.astype(int), 0, Ny - 1),
                         np.clip(xs0.astype(int), 0, Nx - 1)]
        primary = np.column_stack([xs0[keep], ys0[keep]])
        pts.extend(primary)

        if MULTIPLIER_LOCAL:
            jitter   = rng_mul.random((len(primary) * MULTIPLIER_LOCAL, 2)) - 0.5
            expanded = np.repeat(primary, MULTIPLIER_LOCAL, axis=0) + jitter
            keep2 = in_domain[np.clip(expanded[:, 1].astype(int), 0, Ny - 1),
                              np.clip(expanded[:, 0].astype(int), 0, Nx - 1)]
            pts.extend(expanded[keep2])

    pts = np.asarray(pts, float)
    pts = enforce_occupancy(pts, pts)

    # integrate
    for k in range(STEPS):
        pts = one_step(k, pts, rng_step)

    # two normalised column profiles at x = cx ± 8*SCALE
    m = raster(pts)
    cols_x = np.array([cx - 7 * SCALE, cx + 7 * SCALE], dtype=int)
    out = np.zeros((2, Ny), dtype=float)
    for i, x in enumerate(cols_x):
        col = m[:, x].astype(float)
        s = col.sum()
        out[i] = col / s if s else col
    return out  # (2, Ny)

# ---------------- Monte-Carlo loop ----------------
N_RUNS = 200
profiles_sel = np.empty((N_RUNS, 2, Ny), dtype=float)

for i in tqdm(range(N_RUNS), desc='Simulations', unit='run'):
    profiles_sel[i] = run_one(seed=10_000 + i)

# mean ± 95% CI across runs
mean  = profiles_sel.mean(0)                           # (2, Ny)
sem   = profiles_sel.std(0, ddof=1) / np.sqrt(N_RUNS)  # (2, Ny)
ci95  = 1.96 * sem

# y (µm) with reference shift (to match main figure)
REF_SHIFT = 7
y_um = (np.arange(Ny) - (Ny // 2)) / SCALE + REF_SHIFT

# Save  CSVs 
records = []
for run in range(N_RUNS):
    for j, xoff in enumerate([-7, +7]):   # this is where the x position of intensity profile was determined. The value is determined so that lines cross the L-R edge of the cadherin patch
        for yi in range(Ny):
            records.append((y_um[yi], xoff, run + 1, profiles_sel[run, j, yi]))
df_runs = pd.DataFrame(records, columns=['y_um', 'x_offset', 'sample', 'occupancy'])
df_runs.to_csv('ax1_profiles_mc_runs.csv', index=False)

# Wide format: mean ± 95% CI
df_mean = pd.DataFrame({
    'y_um'     : y_um,
    'mean_x-7' : mean[0],
    'mean_x+7' : mean[1],
    'ci95_x-7' : ci95[0],
    'ci95_x+7' : ci95[1],
})
df_mean.to_csv('ax1_profiles_mc_mean_ci.csv', index=False)

print("✓ wrote ax1_profiles_mc_runs.csv and ax1_profiles_mc_mean_ci.csv")